We will start with a dataset of sensor data about hand gestures and, as a way of beginning our ML journey, we will learn a single model (in traditional fashion) from all the data to predict which hand gesture is depicted. Then we will implement Google's federated learning, as above, on this data. Then, there are a number of possible extensions to the work. One is to implement some new ideas that I have had for averaging the models. Another is to move to more challenging datasets from the domain of human activity recognition, which will need use of more complex neural networks for the learning.

do it for users

Reference need to be colleceted
Data needs to be 
maybe bad to have too much accuracy. hard to see if the federated approach makes a diff

===========================================================================
Questions for derek
Do convnets learn patterns from top left and apply it to the bottom right/
	The max pooling concept doesn’t follow that
Diff between rnn and residual connection

===========================================================================
Points to cover after basics

Secure aggregation
    the server doesnt know the key with which the data was encrypted

===========================================================================

Models to make
    traditional model learning from all data
    googles federated learning
        each user builds a model locally and shares the parameters that get learned. Centrally, the parameters are averaged and the averages are sent back to the users, who can incorporate them into the next round of local learning.


===========================================================================
dataset
    https://archive.ics.uci.edu/ml/datasets/MoCap+Hand+Postures


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import train_test_split

Each record is a set. The i-th marker of a given record does not necessarily correspond to the i-th marker of a different record. One may randomly permute the visible (not missing) markers of a given record without changing the set that the record represents. For the sake of convenience, all visible markers of a given record are given a lower index than any missing marker. A class is not guaranteed to have even a single record with all markers visible.

**Information on the Class enumeration**

Ranges from 1 to 5 with:
* 1 = Fist (with thumb out), 
* 2 = Stop (hand flat), 
* 3 = Point1 (point with pointer finger), 
* 4 = Point2 (point with pointer and middle fingers), 
* 5 = Grab (fingers curled as if to grab).

In [2]:
def draw_graphs(history_dict):
    # this is from the book 74,75
    # history = model.fit(...)
    # history_dict = history.history

    loss_values = history_dict['loss']
    val_loss_values = history_dict['val_loss']
    epochs = range(1, len(loss_values) + 1)
    plt.plot(epochs, loss_values, 'bo', label='Training loss')
    plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    plt.clf()
    acc_values = history_dict['acc']
    val_acc_values = history_dict['val_acc']
    plt.plot(epochs, acc_values, 'bo', label='Training acc')
    plt.plot(epochs, val_acc_values, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

In [3]:
def read_file(file):
    "return 2d df after imputing with 0s"
    
    # read data
    df = pd.read_csv("dataset/allUsers.lcl.csv")

    # replace teh question marks with NaN and then change data type to float 32
    df.replace(["?"],np.nan, inplace = True)
    df = df.astype(np.float32)

    # imputation
    df.fillna(0,inplace=True) # fill nulls with 0

    return df 

df = read_file("dataset/allUsers.lcl.csv")

# shuffle the records
df_unshuffled = df
df = df.sample(frac=1) # shuffle data


In [4]:

def split_dataframe(df, user=None, val_ratio = 0.25, test_ratio =  0.75, seed = 1):
    # split into train, validation and test data using sklearn and return dfs for each
    if user!=None:
        df = df[df["User"] == user]
    if df.shape[0] == 0:
        print(f"Dataframe for user {user} is of shape {df.shape}, no data. Skipping...")
        df = pd.DataFrame()
        return df, df, df, df, df, df, df, df, df
        
    df_train, df_test = train_test_split(df,
                                         test_size=test_ratio, 
                                         random_state = seed)

    df_train, df_val  = train_test_split(df_train, 
                                         test_size=val_ratio,
                                         random_state = seed)

    # store class and user information (in order)
    df_val_class, df_train_class, df_test_class = df_val["Class"], df_train["Class"], df_test["Class"]
    df_val_user,  df_train_user,  df_test_user  = df_val["User"],  df_train["User"],  df_test["User"]

    # drop the class and user identifier columns from data frame
    df_val   = df_val.  drop(df_train.columns[[0,1]], axis=1)
    df_train = df_train.drop(df_train.columns[[0,1]], axis=1)
    df_test  = df_test. drop(df_test. columns[[0,1]], axis=1)
    
    return df_val, df_val_class,  df_val_user, \
           df_test, df_test_class, df_test_user, \
           df_train, df_train_class, df_train_user

https://www.tensorflow.org/beta/tutorials/keras/basic_classification

In [5]:
def train_model(train_data, train_class,
          val_data, val_class, 
          epochs=32, verbose=0, return_weights = False, init_seed = 1):
    
    # same seed value for consistency sake, across all trainings too
    
    model = keras.Sequential([
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation='relu',
                           kernel_initializer=keras.initializers.glorot_uniform(seed=init_seed)),
        keras.layers.Dense(32, activation='relu',
                           kernel_initializer=keras.initializers.glorot_uniform(seed=init_seed)),
        keras.layers.Dense(6, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    history = model.fit(
        train_data, 
        train_class, 
        epochs = epochs,
        verbose = verbose,
        validation_data = (val_data, val_class)
    )

    if return_weights:
        return model, history, model.get_weights()
    return model, history


In [19]:


weights = []
# for i in range(6):
   
for i in range(6):
    print(f"User {i} being trained on the model...")
    
    df_val, df_val_class,  df_val_user, df_test, df_test_class, df_test_user, df_train, df_train_class, df_train_user = split_dataframe(df,user=i)
    if df_val.shape[0]==0:
        continue
    
    model, history, weight = train_model(
        df_train.values, 
        df_train_class.values, 
        df_val.values, 
        df_val_class.values,
        epochs = 16,
        return_weights=True,
        init_seed=1
    )
   
    weights.append(weight)
#     draw_graphs(history.history)
#     print(f"User {i} done!")
    

User 0 being trained on the model...
User 1 being trained on the model...
User 2 being trained on the model...
User 3 being trained on the model...
Dataframe for user 3 is of shape (0, 38), no data. Skipping...
User 4 being trained on the model...
User 5 being trained on the model...


In [25]:
# the weights are a list of numpy arrays from different layers of the model
# weights = [ [user0] [user1] [] [list of numpy arrays of layer weights and biases] [] []]


def average_weights(weights):
    new_weights = []

    for layer_index in range(len(weights[0])):
        temp_layer_data = None
        for i, weight_user in enumerate(weights):
            if temp_layer_data is None:
                temp_layer_data = weight_user[layer_index]
                continue
            temp_layer_data += weight_user[layer_index]
        new_weights.append(temp_layer_data/(i+1))
    return new_weights

new_weights = average_weights(weights)

In [29]:
def evaluate_all_users(new_weights, df, users_to_ignore=[3]):
    for i in range(len(new_weights)):
        if i in users_to_ignore:
            continue
        df_val, df_val_class,  df_val_user, df_test, df_test_class, df_test_user, df_train, df_train_class, df_train_user = split_dataframe(df,user=i)
        print(model.evaluate(df_test.values, df_test_class))
    

In [33]:
# model.summary()


In [32]:
evaluate_all_users(new_weights, df)

6787/6787 [==============================] - 0s 34us/sample - loss: 23.8358 - acc: 0.2145
[23.835775820376753, 0.21452777]
3538/3538 [==============================] - 0s 30us/sample - loss: 19.2764 - acc: 0.3143
[19.276366829535192, 0.31430188]
3385/3385 [==============================] - 0s 33us/sample - loss: 21.9016 - acc: 0.1799
[21.901599764013923, 0.17991138]
285/285 [==============================] - 0s 50us/sample - loss: 19.4395 - acc: 0.3439
[19.439500815408273, 0.34385964]
3829/3829 [==============================] - 0s 39us/sample - loss: 18.1693 - acc: 0.3682
[18.16926274233619, 0.36824235]


In [34]:
model.set_weights(new_weights)
evaluate_all_users(new_weights, df)

6787/6787 [==============================] - 0s 35us/sample - loss: 23.8358 - acc: 0.2145
[23.835775820376753, 0.21452777]
3538/3538 [==============================] - 0s 36us/sample - loss: 19.2764 - acc: 0.3143
[19.276366829535192, 0.31430188]
3385/3385 [==============================] - 0s 32us/sample - loss: 21.9016 - acc: 0.1799
[21.901599764013923, 0.17991138]
285/285 [==============================] - 0s 46us/sample - loss: 19.4395 - acc: 0.3439
[19.439500815408273, 0.34385964]
3829/3829 [==============================] - 0s 37us/sample - loss: 18.1693 - acc: 0.3682
[18.16926274233619, 0.36824235]
